In [1]:
import pyprob
%matplotlib inline
import matplotlib.pyplot as plt
from pyprob import Model
import numpy as np

import math
import pyprob
from pyprob import Model
from pyprob.distributions import Normal, Uniform, Categorical
import torch


ModuleNotFoundError: No module named 'pyprob'

In [2]:
class MarkovChainPath(Model):
    def __init__(self):
        super().__init__(name="Gaussian with unknown mean") # give the model a name

    def forward(self): # Needed to specifcy how the generative model is run forward
        # sample the (latent) mean variable to be inferred:
        coords = [[0,0]]
        moves = {0: -1, 1: 1, 2: 1}
        for i in range(1,5):
            last = coords[-1][1]
            move = pyprob.sample(Categorical([1/2.,1/2.]), name = 'input{}'.format(i))
            move = moves[move.item()]
            coords.append([i,last+move])

        obs_distr = Normal(coords[-1][1], 0.1)
        pyprob.observe(obs_distr, name='obs0') # NOTE: observe -> denotes observable variables
        return coords

model = MarkovChainPath()

In [ ]:
model.learn_inference_network(
    num_traces=50000,
    observe_embeddings={'obs0': {'dim': 100, 'depth': 5}}
)

Creating new inference network...
Observable obs0: observe embedding not specified, using the default FEEDFORWARD.
Observe embedding dimension: 100
Train. time | Epoch| Trace     | Init. loss| Min. loss | Curr. loss| T.since min | Traces/sec
New layers, address: 74__forward__move__Categorical(len_probs:2)__1, distribution: Categorical
New layers, address: 74__forward__move__Categorical(len_probs:2)__2, distribution: Categorical
New layers, address: 74__forward__move__Categorical(len_probs:2)__3, distribution: Categorical
New layers, address: 74__forward__move__Categorical(len_probs:2)__4, distribution: Categorical
Total addresses: 4, parameters: 54,070


In [ ]:
prior = model.prior_traces(
    num_traces=1000,
)

In [ ]:
condition = {'obs0': 0}
posterior = model.posterior_traces(
    num_traces=1000,
    inference_engine=pyprob.InferenceEngine.IMPORTANCE_SAMPLING_WITH_INFERENCE_NETWORK,
    observe=condition
)

In [ ]:
post_paths  = [posterior.sample().result for i in range(1000)]
prior_paths = [prior.sample().result for i in range(1000)]

In [ ]:
for p in prior_paths:
    p = np.asarray(p)
    plt.plot(p[:,0],p[:,1], c = 'orangered', alpha = 0.1)    
    
    
for p in post_paths:
    p = np.asarray(p)
    plt.plot(p[:,0],p[:,1] + 0.5, c = 'steelblue', alpha = 0.1, )#linewidth = 3)

In [ ]:
plt.hist([p[-1][1] for p in prior_paths], np.linspace(-15.5,15.5,32), alpha = 0.2);
plt.hist([p[-1][1] for p in post_paths], np.linspace(-15.5,15.5,32), alpha = 0.2);

In [ ]:
plt.hist([t.log_importance_weight for t in posterior], bins = 100);